In [ ]:
# === Cell 1: Unified Environment & Project-Wide Setup ===
import os, json, math, datetime as dt
from datetime import datetime  # for human-readable prints
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional (map preview): keep if you use geemap
try:
    import geemap
    GEEMAP_AVAILABLE = True
except Exception:
    GEEMAP_AVAILABLE = False

plt.style.use('seaborn-v0_8-whitegrid')
os.makedirs('outputs', exist_ok=True)

import ee

# --- Earth Engine authentication and initialization ---
# You can override with: export EE_PROJECT_ID=your-project-id
EE_PROJECT_ID = os.environ.get('EE_PROJECT_ID', 'nasa-flood')  # <— 원하는 기본 프로젝트 ID
def _ee_init():
    try:
        ee.Initialize()
        return "Initialized with default credentials"
    except Exception:
        try:
            ee.Initialize(project=EE_PROJECT_ID)
            return f"Initialized with project='{EE_PROJECT_ID}'"
        except Exception:
            print("Authenticating with Earth Engine...")
            ee.Authenticate()
            ee.Initialize(project=EE_PROJECT_ID)
            return f"Authenticated & initialized with project='{EE_PROJECT_ID}'"

print(_ee_init())
print(f"Current time: {datetime.now().isoformat(timespec='seconds')}")

# ===== Project-wide constants =====
CFG = {
    # AOIs (EPSG:4326)
    # Mekong Delta (VN lower basin); box chosen for stability/reproducibility
    "AOI_DELTA": ee.Geometry.Rectangle([104.30,  8.50, 106.90, 10.90], geodesic=False),
    # Tonlé Sap (KH); "Mekong's heartbeat"
    "AOI_TONLESAP": ee.Geometry.Rectangle([103.30, 12.00, 105.20, 13.70], geodesic=False),

    # Analysis windows
    "YEARS": list(range(2015, 2025)),
    "FLOOD_MONTHS": (8, 9),     # Aug–Sep (wet-season peak)
    "DROUGHT_MONTHS": (3, 4),   # Mar–Apr (dry-season trough)

    # Thresholds (physical/empirical, fixed for comparability)
    "TH_VV_DB": -16.0,
    "TH_VH_DB": -22.0,

    # Landsat5 baseline window (pre-major-dam reference)
    "BASELINE_YEARS": [2005, 2006, 2007, 2008],

    # Event markers for plots
    "EVENTS": {
        "JINGHONG_FLOW_CUT": "2019-07-15",  # Jinghong flow cut (smoking gun)
        "XIAOWAN_ONLINE":    "2009-01-01",
        "NUOZHADU_ONLINE":   "2012-01-01"
    }
}

# ===== Utilities for Earth Engine =====
def _daterange_of_year_months(year:int, m1:int, m2:int):
    """Return ISO start and inclusive end-of-month last day for [m1..m2]."""
    start = dt.date(year, m1, 1)
    if m2 == 12:
        end = dt.date(year+1, 1, 1) - dt.timedelta(days=1)
    else:
        end = dt.date(year, m2+1, 1) - dt.timedelta(days=1)
    return start.isoformat(), end.isoformat()

def s1_min(aoi, start, end, pol):
    """Min-composite Sentinel-1 GRD over period to stabilize speckle."""
    return (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(aoi)
            .filterDate(start, end)
            .filter(ee.Filter.eq('instrumentMode','IW'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
            .select(pol)
            .min()
            .clip(aoi))

def classify_water(img_min, pol, threshold_db):
    """Water = backscatter < threshold (in dB); returns self-masked binary."""
    return img_min.lt(threshold_db).selfMask()

def area_km2(mask_img, aoi, scale=30, band_name='constant'):
    """Compute km² of a self-masked image."""
    area = (mask_img.multiply(ee.Image.pixelArea())
            .reduceRegion(ee.Reducer.sum(), aoi, scale, maxPixels=1e12))
    # selfMask() produces band 'constant'
    return ee.Number(area.get(band_name)).divide(1e6)

def landsat5_c2_sr_mask_scale(img):
    """L5 C2 L2 scaling + cloud/shadow masking."""
    qa = img.select('QA_PIXEL')
    cloud  = 1 << 3
    shadow = 1 << 4
    mask = qa.bitwiseAnd(cloud).eq(0).And(qa.bitwiseAnd(shadow).eq(0))
    optical = img.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal = img.select('ST_B6').multiply(0.00341802).add(149.0)
    return (img.addBands(optical, None, True)
               .addBands(thermal, None, True)
               .updateMask(mask))

def landsat5_median(aoi, start_date, end_date, months=None):
    """Median composite with optional wrapped-month filter ((11,12),(1,4))."""
    col = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
           .filterBounds(aoi)
           .filterDate(start_date, end_date)
           .map(landsat5_c2_sr_mask_scale))
    if months:
        # e.g., months=((11,12),(1,4)) to handle dry-season wrap
        f = ee.Filter.Or(
            ee.Filter.calendarRange(int(months[0][0]), int(months[0][1]), 'month'),
            ee.Filter.calendarRange(int(months[1][0]), int(months[1][1]), 'month')
        )
        col = col.filter(f)
    return col.median().clip(aoi)

def mndwi(img):  # SR_B2 (Green), SR_B5 (SWIR)
    return img.normalizedDifference(['SR_B2','SR_B5']).rename('MNDWI')

def water_mask_from_mndwi(img, threshold=0.0):
    return img.gt(threshold).selfMask()

def chirps_sum_mm(aoi, start, end):
    """Return AOI-mean of CHIRPS precipitation sum (mm) over [start,end]."""
    col = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
           .filterBounds(aoi).filterDate(start, end).select('precipitation'))
    total = col.sum().reduceRegion(ee.Reducer.mean(), aoi, 5000, maxPixels=1e12)
    return ee.Number(total.get('precipitation'))

# (Optional) quick summary print
print(f"AOI_DELTA bounds: {CFG['AOI_DELTA'].bounds().getInfo()['coordinates'][0][0]} …")
print(f"AOI_TONLESAP bounds: {CFG['AOI_TONLESAP'].bounds().getInfo()['coordinates'][0][0]} …")
print("Setup complete ✅")


In [ ]:
# === Cell 2: Load pre-dam baselines computed in 01 ===
baseline = json.load(open('outputs/baseline_summary.json', 'r', encoding='utf-8'))

BASE_WET_DELTA = next(a['baseline_wet_km2'] for a in baseline['areas'] if a['aoi']=='Mekong_Delta')
BASE_WET_TS    = next(a['baseline_wet_km2'] for a in baseline['areas'] if a['aoi']=='Tonle_Sap')

print("Loaded baselines:")
print("- Delta (wet):", f"{BASE_WET_DELTA:,.2f} km²")
print("- Tonle Sap (wet):", f"{BASE_WET_TS:,.2f} km²")


In [ ]:
# === Cell 3: Annual flood extent (Aug–Sep) for 2015–2024, VV & VH, by AOI ===

def flood_extent_for_year(aoi, year, th_vv=CFG['TH_VV_DB'], th_vh=CFG['TH_VH_DB']):
    start, end = _daterange_of_year_months(year, *CFG['FLOOD_MONTHS'])
    vv_min = s1_min(aoi, start, end, 'VV')
    vh_min = s1_min(aoi, start, end, 'VH')

    vv_mask = classify_water(vv_min, 'VV', th_vv)   # open water 중심
    vh_mask = classify_water(vh_min, 'VH', th_vh)   # flooded vegetation 포함

    vv_km2 = float(area_km2(vv_mask, aoi).getInfo() or 0.0)
    vh_km2 = float(area_km2(vh_mask, aoi).getInfo() or 0.0)

    # VH only (식생 하부 침수)
    vh_only = vh_mask.updateMask(vv_mask.Not())
    vh_only_km2 = float(area_km2(vh_only, aoi).getInfo() or 0.0)

    return {
        "year": year,
        "vv_km2": vv_km2,
        "vh_km2": vh_km2,
        "vh_only_km2": vh_only_km2,
        "vh_gain_pct_over_vv": (vh_km2 - vv_km2) / vv_km2 * 100 if vv_km2 > 0 else np.nan
    }

rows_delta, rows_ts = [], []
for y in CFG['YEARS']:
    rows_delta.append(flood_extent_for_year(CFG['AOI_DELTA'], y))
    rows_ts.append(flood_extent_for_year(CFG['AOI_TONLESAP'], y))

df_delta = pd.DataFrame(rows_delta)
df_ts    = pd.DataFrame(rows_ts)

# 열 이름 가독성
df_delta = df_delta.rename(columns={
    "vv_km2":"delta_vv_km2", "vh_km2":"delta_vh_km2",
    "vh_only_km2":"delta_vh_only_km2", "vh_gain_pct_over_vv":"delta_vh_gain_pct"
})
df_ts = df_ts.rename(columns={
    "vv_km2":"ts_vv_km2", "vh_km2":"ts_vh_km2",
    "vh_only_km2":"ts_vh_only_km2", "vh_gain_pct_over_vv":"ts_vh_gain_pct"
})

df = pd.merge(df_delta, df_ts, on="year")
display(df.head())

# 저장
df.to_csv("outputs/annual_flood_by_aoi_2015_2024.csv", index=False)
print("Saved -> outputs/annual_flood_by_aoi_2015_2024.csv")


In [ ]:
# === Cell 4: Plots with baselines & event marker ===
events = {k: pd.to_datetime(v) for k,v in CFG["EVENTS"].items()}
jinghong_year = 2019  # 연 단위 플롯이라 2019 수직선만 표시

def plot_flood_series(ax, years, vv, vh, baseline_value, title, label_prefix):
    ax.plot(years, vv, marker='o', linestyle='-', label=f'{label_prefix} VV (open water)')
    ax.plot(years, vh, marker='s', linestyle='--', label=f'{label_prefix} VH (incl. flooded veg.)')
    ax.axhline(y=baseline_value, color='firebrick', linestyle='--', linewidth=2, 
               label='Pre-dam wet baseline (2005–2008)')
    ax.axvline(x=jinghong_year, color='gray', linestyle=':', linewidth=2,
               label='2019 Jinghong flow cut')

    ax.set_title(title, fontsize=15)
    ax.set_xlabel("Year")
    ax.set_ylabel("Flood extent (km²)")
    ax.grid(True)
    ax.legend(loc='best')

fig, axes = plt.subplots(1, 2, figsize=(15,6), sharex=True)
plot_flood_series(axes[0], df['year'], df['delta_vv_km2'], df['delta_vh_km2'],
                  BASE_WET_DELTA, 'Mekong Delta – Aug–Sep flood extent', 'Delta')
plot_flood_series(axes[1], df['year'], df['ts_vv_km2'], df['ts_vh_km2'],
                  BASE_WET_TS, 'Tonlé Sap – Aug–Sep flood extent', 'Tonlé Sap')

plt.tight_layout()
plt.savefig("outputs/annual_flood_trend_panels.png", dpi=200)
plt.show()
print("Saved -> outputs/annual_flood_trend_panels.png")


In [ ]:
# === Cell 5: Tech highlight (VH gain) summary ===
summary = pd.DataFrame({
    "AOI": ["Mekong_Delta","Tonle_Sap"],
    "mean_vh_gain_pct": [
        df['delta_vh_gain_pct'].replace([np.inf, -np.inf], np.nan).dropna().mean(),
        df['ts_vh_gain_pct'].replace([np.inf, -np.inf], np.nan).dropna().mean()
    ],
    "sum_vh_only_km2": [
        df['delta_vh_only_km2'].sum(),
        df['ts_vh_only_km2'].sum()
    ]
})
display(summary.round(2))
summary.to_csv("outputs/vh_gain_summary.csv", index=False)
print("Saved -> outputs/vh_gain_summary.csv")


In [ ]:
# === Cell 6: (Optional) 2018 Map – VV vs VH-only ===
if GEEMAP_AVAILABLE:
    year_to_show = 2018
    start, end = _daterange_of_year_months(year_to_show, *CFG['FLOOD_MONTHS'])

    # Delta 예시
    vv_min_d = s1_min(CFG['AOI_DELTA'], start, end, 'VV')
    vh_min_d = s1_min(CFG['AOI_DELTA'], start, end, 'VH')
    vv_mask_d = classify_water(vv_min_d, 'VV', CFG['TH_VV_DB'])
    vh_mask_d = classify_water(vh_min_d, 'VH', CFG['TH_VH_DB'])
    vh_only_d = vh_mask_d.updateMask(vv_mask_d.Not())

    m = geemap.Map(center=[10.2, 105.5], zoom=7)
    m.addLayer(vv_mask_d, {'palette':['#9bd3f7']}, f'{year_to_show} Delta – VV open water')
    m.addLayer(vh_only_d, {'palette':['#08306b']}, f'{year_to_show} Delta – VH-only flooded veg.')
    m.addLayerControl()
    m

    # 필요시 GeoTIFF export
    geemap.ee_export_image(vv_mask_d, filename=f'outputs/delta_vv_mask_{year_to_show}.tif', scale=30, region=CFG['AOI_DELTA'])
    geemap.ee_export_image(vh_only_d, filename=f'outputs/delta_vh_only_{year_to_show}.tif', scale=30, region=CFG['AOI_DELTA'])
    print("GeoTIFF saved for 2018 example (Delta).")
else:
    print("geemap not available: skip interactive map/exports.")


In [ ]:
# === Cell 7: (Optional) Dashboard payload ===
payload = {
    "years": df["year"].tolist(),
    "delta": {
        "vv_km2": df["delta_vv_km2"].round(1).tolist(),
        "vh_km2": df["delta_vh_km2"].round(1).tolist(),
        "vh_gain_pct": df["delta_vh_gain_pct"].round(2).replace([np.inf, -np.inf], np.nan).fillna(0).tolist()
    },
    "tonle_sap": {
        "vv_km2": df["ts_vv_km2"].round(1).tolist(),
        "vh_km2": df["ts_vh_km2"].round(1).tolist(),
        "vh_gain_pct": df["ts_vh_gain_pct"].round(2).replace([np.inf, -np.inf], np.nan).fillna(0).tolist()
    },
    "baselines": {
        "delta_wet_km2": round(BASE_WET_DELTA, 1),
        "tonle_wet_km2": round(BASE_WET_TS, 1)
    },
    "events": CFG["EVENTS"]
}
with open("outputs/annual_flood_dashboard.json", "w", encoding="utf-8") as f:
    json.dump(payload, f, ensure_ascii=False, indent=2)
print("Saved -> outputs/annual_flood_dashboard.json")
